# Do the steady states have large weights in W_out?

In [2]:
%run "../HyperParameterOpt/GenerateExperiments/res_experiment.py"
import copy
from matplotlib import pyplot as plt
import scipy as sp
from scipy import signal
DIFF_EQ_PARAMS = {
                  "x0": [-20, 10, -.5],
                  "begin": 0,
                  "end": 85,
                  "timesteps": 85000,
                  "train_per": .889,
                  "solver": lorenz_equ,
                  "clip": 40
                 }

RES_PARAMS = {
              "uniform_weights": True,
              "solver": "ridge",
              "ridge_alpha": 1e-6,
              "signal_dim": 3,
              "network": "random graph",

              "res_sz": 2500,
              "activ_f": np.tanh,
              "connect_p": .1,
              "spect_rad": 5,
              "gamma": 5,
              "sigma": .14,
              "sparse_res": True,
             }

TOL = 5
plt.rcParams["figure.figsize"] = [10, 5]

In [3]:
def adj_matrix(topology, topo_params, n, remove_p=0):
    adj = generate_adj(topology, topo_params, n)
    adj = remove_edges(adj, floor(remove_p*np.sum(adj != 0)))
    return adj

In [4]:
def kuramoto_order(peaktimes, t):
    """ Compute the kuramoto order parameter of a system of oscilators.
        Parameters:
        ----------
        peaktimes (list): A list of length N > 1 where each entry is an array of 
                          oscilator peak times (use scipy.signal.find_peaks_cwt)
        t (ndarry): Time values where the order parameter should be computed
        Returns
        -------
        R (ndarray): kuramoto order parameter for each of the timesteps in t
    """
    R = np.zeros(len(t), np.complex)
    timemask = np.argsort(t)
    for peaks in peaktimes:
        prevpeak = peaks[1]
        prevprevpeak = peaks[0]
        j = 2
        for i, ti in enumerate(t[timemask]):
            if j >= len(peaks) - 1: break
            # Find first t value that occurs after the first peak
            if ti < prevpeak:
                pass
            # Find the first peak that is greater than the time value
            if ti >= peaks[j]:
                prevprevpeak = prevpeak
                prevpeak = peaks[j]
                j += 1
            # Compute kuramoto order
            if (ti >= prevpeak) and (ti < peaks[j]):
                phi = 2 * np.pi *(ti - prevpeak) / (prevpeak - prevprevpeak)
                R[i] += np.exp(1j*phi)
    R = np.abs(R) / len(peaktimes)
    return R

def state_peaks(t, states):
    peaks = []
    for i in range(states.shape[1]):
        peakidx = sp.signal.find_peaks(states[:,i].T)[0]
        peaktimes = t[peakidx]
        peaks.append(peaktimes)
    return peaks

In [5]:
def steady_state_ratio(topo, topo_p, remove_p=0.0):
    DIFF_EQ_PARAMS["x0"] = random_lorenz_x0()
    train_t, test_t, u = rc_solve_ode(DIFF_EQ_PARAMS)
    A = adj_matrix(topo, topo_p, 2500, remove_p=remove_p)
    rc = ResComp(A, **RES_PARAMS)
    r0 = rc.state_0
    states = rc.drive(train_t, u)
    mask = np.abs(np.mean(states, axis=0)) > .97
    rc.state_0 = r0
    rc.fit(train_t, u)
    mean_ss = np.mean(np.abs(rc.W_out[:, mask]))
    mean_os = np.mean(np.abs(rc.W_out[:, ~mask]))
    n = sum(mask)
    return np.round(mean_ss, 4), np.round(mean_os,4), n

def rep_ss_ratio(topo, topo_p=None, remove_p=0.0, N=10):
    totss = 0
    totos = 0
    for i in range(N):
        ss, os, n = steady_state_ratio(topo, topo_p, remove_p=remove_p)
        totss += ss
        totos += os
        print(f"Trial {i}: {n} Steady states. Avg weight={ss} Oscil. w avg={os} Ratio={ss/os}")
    print(f"Average over all trials: ss={totss/N} os={totos/N} Ratio={totss/totos}")

1. Count total contribution of steady states vs. oscillatory states
2. Factor in the percentage shift of oscil v. steady states

## Watts (Remove 0%)

In [30]:
rep_ss_ratio("watts4", topo_p=0.1, remove_p=0.0)

Trial 0: 2411 Steady states. Avg weight=15.2134 Oscil. w avg=63.0096 Ratio=0.2414457479495188
Trial 1: 2397 Steady states. Avg weight=14.5715 Oscil. w avg=73.0028 Ratio=0.19960193307653956
Trial 2: 2427 Steady states. Avg weight=24.5441 Oscil. w avg=81.6604 Ratio=0.3005630636146774
Trial 3: 2395 Steady states. Avg weight=15.0349 Oscil. w avg=64.9498 Ratio=0.2314849314393578
Trial 4: 2399 Steady states. Avg weight=12.3745 Oscil. w avg=57.6731 Ratio=0.2145627684310363
Trial 5: 2386 Steady states. Avg weight=12.5761 Oscil. w avg=49.9826 Ratio=0.2516095601269242
Trial 6: 2415 Steady states. Avg weight=15.0206 Oscil. w avg=59.5692 Ratio=0.25215379760010204
Trial 7: 2402 Steady states. Avg weight=12.4572 Oscil. w avg=50.1084 Ratio=0.24860502430730175
Trial 8: 2397 Steady states. Avg weight=13.5815 Oscil. w avg=50.8949 Ratio=0.266853849796345
Trial 9: 2360 Steady states. Avg weight=7.7841 Oscil. w avg=34.9392 Ratio=0.2227898749828273
Average over all trials: ss=14.315789999999998 os=58.578999

## Watts (Remove 90%)

In [31]:
rep_ss_ratio("watts4", topo_p=0.1, remove_p=0.9)

Trial 0: 314 Steady states. Avg weight=2.8965 Oscil. w avg=3.6699 Ratio=0.7892585629036213
Trial 1: 338 Steady states. Avg weight=2.8721 Oscil. w avg=3.1751 Ratio=0.904569934805203
Trial 2: 298 Steady states. Avg weight=2.4407 Oscil. w avg=3.9501 Ratio=0.6178830915673021
Trial 3: 305 Steady states. Avg weight=3.1239 Oscil. w avg=3.6686 Ratio=0.8515237420269313
Trial 4: 327 Steady states. Avg weight=3.3173 Oscil. w avg=3.868 Ratio=0.8576266804550156
Trial 5: 325 Steady states. Avg weight=3.3023 Oscil. w avg=3.8546 Ratio=0.8567166502360815
Trial 6: 317 Steady states. Avg weight=3.5127 Oscil. w avg=3.942 Ratio=0.8910958904109589
Trial 7: 321 Steady states. Avg weight=3.2068 Oscil. w avg=3.5143 Ratio=0.912500355689611
Trial 8: 264 Steady states. Avg weight=3.3616 Oscil. w avg=3.8801 Ratio=0.8663694234684673
Trial 9: 232 Steady states. Avg weight=3.1498 Oscil. w avg=3.6115 Ratio=0.8721583829433753
Average over all trials: ss=3.1183699999999996 os=3.71342 Ratio=0.8397568817962956


## Geometric (Remove 0%)

In [32]:
rep_ss_ratio("geom", topo_p=4, remove_p=0.0)

Trial 0: 844 Steady states. Avg weight=0.8261 Oscil. w avg=4.648 Ratio=0.17773235800344234
Trial 1: 939 Steady states. Avg weight=1.0829 Oscil. w avg=5.5504 Ratio=0.19510305563562988
Trial 2: 954 Steady states. Avg weight=0.9376 Oscil. w avg=5.1595 Ratio=0.18172303517782729
Trial 3: 861 Steady states. Avg weight=0.8596 Oscil. w avg=4.8279 Ratio=0.1780484268522546
Trial 4: 904 Steady states. Avg weight=0.845 Oscil. w avg=4.8192 Ratio=0.1753403054448871
Trial 5: 963 Steady states. Avg weight=0.8902 Oscil. w avg=4.7655 Ratio=0.18680096527122023
Trial 6: 699 Steady states. Avg weight=0.805 Oscil. w avg=4.2963 Ratio=0.1873705281288551
Trial 7: 929 Steady states. Avg weight=1.1324 Oscil. w avg=5.5178 Ratio=0.20522672079451956
Trial 8: 718 Steady states. Avg weight=0.9502 Oscil. w avg=4.533 Ratio=0.20961835429075668
Trial 9: 885 Steady states. Avg weight=0.7841 Oscil. w avg=4.3663 Ratio=0.17957996472986282
Average over all trials: ss=0.9113100000000001 os=4.84839 Ratio=0.18796136449419293


## Geometric (Remove 90%)

In [33]:
rep_ss_ratio("geom", topo_p=4.0, remove_p=0.9)

Trial 0: 303 Steady states. Avg weight=3.2524 Oscil. w avg=3.5181 Ratio=0.9244762798101248
Trial 1: 322 Steady states. Avg weight=2.6181 Oscil. w avg=3.6911 Ratio=0.7093007504537943
Trial 2: 258 Steady states. Avg weight=3.0602 Oscil. w avg=3.5431 Ratio=0.8637069233157405
Trial 3: 294 Steady states. Avg weight=2.6215 Oscil. w avg=3.4386 Ratio=0.7623742220671204
Trial 4: 272 Steady states. Avg weight=3.0061 Oscil. w avg=3.7564 Ratio=0.800260888084336
Trial 5: 258 Steady states. Avg weight=3.1031 Oscil. w avg=3.5273 Ratio=0.8797380432625521
Trial 6: 322 Steady states. Avg weight=2.8545 Oscil. w avg=3.9008 Ratio=0.7317729696472518
Trial 7: 315 Steady states. Avg weight=3.6186 Oscil. w avg=3.7562 Ratio=0.9633672328417017
Trial 8: 244 Steady states. Avg weight=3.0727 Oscil. w avg=3.5338 Ratio=0.8695172335729244
Trial 9: 324 Steady states. Avg weight=2.8254 Oscil. w avg=3.6619 Ratio=0.7715666730385865
Average over all trials: ss=3.00326 os=3.63273 Ratio=0.8267226025606087


## Identity (Remove 0%)

In [34]:
rep_ss_ratio("ident", topo_p=None, remove_p=0.0)

Trial 0: 2487 Steady states. Avg weight=308.5484 Oscil. w avg=10.9237 Ratio=28.245777529591624
Trial 1: 2484 Steady states. Avg weight=287.864 Oscil. w avg=4.3714 Ratio=65.85167223315185
Trial 2: 2495 Steady states. Avg weight=298.4212 Oscil. w avg=1.6849 Ratio=177.11508101371
Trial 3: 2485 Steady states. Avg weight=312.7512 Oscil. w avg=3.6511 Ratio=85.65944509873736
Trial 4: 2482 Steady states. Avg weight=308.5008 Oscil. w avg=22.7208 Ratio=13.577902186542728
Trial 5: 2471 Steady states. Avg weight=296.9365 Oscil. w avg=75.4506 Ratio=3.935508796484058
Trial 6: 2484 Steady states. Avg weight=312.8132 Oscil. w avg=97.5122 Ratio=3.207939109157623
Trial 7: 2492 Steady states. Avg weight=302.0613 Oscil. w avg=0.3669 Ratio=823.2796402289453
Trial 8: 2488 Steady states. Avg weight=298.726 Oscil. w avg=2.8021 Ratio=106.60790121694444
Trial 9: 2479 Steady states. Avg weight=306.523 Oscil. w avg=7.7163 Ratio=39.72409056153856
Average over all trials: ss=303.31456 os=22.72 Ratio=13.350112676056

## Identity (Remove 90%)

In [35]:
rep_ss_ratio("ident", topo_p=None, remove_p=0.9)

Trial 0: 248 Steady states. Avg weight=4.4763 Oscil. w avg=7.9126 Ratio=0.565717968809241
Trial 1: 246 Steady states. Avg weight=11.354 Oscil. w avg=12.6688 Ratio=0.8962174791614044
Trial 2: 249 Steady states. Avg weight=4.2036 Oscil. w avg=8.1204 Ratio=0.517659228609428
Trial 3: 249 Steady states. Avg weight=4.2171 Oscil. w avg=7.8276 Ratio=0.5387475088149625
Trial 4: 246 Steady states. Avg weight=6.6377 Oscil. w avg=9.0348 Ratio=0.7346814539336786
Trial 5: 249 Steady states. Avg weight=3.8256 Oscil. w avg=7.9874 Ratio=0.4789543531061422
Trial 6: 249 Steady states. Avg weight=6.5791 Oscil. w avg=8.6798 Ratio=0.7579782944307473
Trial 7: 249 Steady states. Avg weight=2.4734 Oscil. w avg=7.0186 Ratio=0.35240646282734445
Trial 8: 247 Steady states. Avg weight=4.5476 Oscil. w avg=7.9026 Ratio=0.5754561789790702
Trial 9: 247 Steady states. Avg weight=6.5382 Oscil. w avg=9.4205 Ratio=0.6940395945013533
Average over all trials: ss=5.485259999999999 os=8.65731 Ratio=0.6335986582437269


## Barabasi (Remove 0%)

In [37]:
rep_ss_ratio("barab2", topo_p=None, remove_p=0.0)

Trial 0: 133 Steady states. Avg weight=0.8156 Oscil. w avg=4.0335 Ratio=0.20220652039171935
Trial 1: 209 Steady states. Avg weight=0.8448 Oscil. w avg=4.0797 Ratio=0.20707404956246783
Trial 2: 78 Steady states. Avg weight=1.3067 Oscil. w avg=4.8881 Ratio=0.26732268161453326
Trial 3: 247 Steady states. Avg weight=1.1088 Oscil. w avg=4.9253 Ratio=0.2251233427405437
Trial 4: 162 Steady states. Avg weight=1.247 Oscil. w avg=4.8018 Ratio=0.259694281311175
Trial 5: 192 Steady states. Avg weight=0.7589 Oscil. w avg=4.952 Ratio=0.15325121163166397
Trial 6: 135 Steady states. Avg weight=0.7749 Oscil. w avg=4.2959 Ratio=0.18038129379175497
Trial 7: 130 Steady states. Avg weight=1.0755 Oscil. w avg=4.085 Ratio=0.26328029375764994
Trial 8: 138 Steady states. Avg weight=1.0801 Oscil. w avg=4.6761 Ratio=0.23098308419409339
Trial 9: 277 Steady states. Avg weight=0.8835 Oscil. w avg=4.6694 Ratio=0.18921060521694433
Average over all trials: ss=0.9895799999999999 os=4.54068 Ratio=0.21793652052115542


## Barabasi (Remove 90%)

In [38]:
rep_ss_ratio("barab2", topo_p=None, remove_p=0.9)

Trial 0: 221 Steady states. Avg weight=3.1927 Oscil. w avg=3.1866 Ratio=1.0019142659888283
Trial 1: 216 Steady states. Avg weight=3.9784 Oscil. w avg=3.2349 Ratio=1.2298370892454171
Trial 2: 284 Steady states. Avg weight=3.6799 Oscil. w avg=3.7595 Ratio=0.9788269716717648
Trial 3: 310 Steady states. Avg weight=4.0254 Oscil. w avg=3.6574 Ratio=1.1006179253021273
Trial 4: 279 Steady states. Avg weight=3.0991 Oscil. w avg=3.6187 Ratio=0.8564125238345262
Trial 5: 198 Steady states. Avg weight=3.9259 Oscil. w avg=3.523 Ratio=1.1143627590122054
Trial 6: 255 Steady states. Avg weight=3.643 Oscil. w avg=3.5482 Ratio=1.0267177723916352
Trial 7: 187 Steady states. Avg weight=3.6917 Oscil. w avg=3.5696 Ratio=1.03420551322277
Trial 8: 183 Steady states. Avg weight=4.0932 Oscil. w avg=4.0552 Ratio=1.0093706845531665
Trial 9: 295 Steady states. Avg weight=3.6596 Oscil. w avg=3.2188 Ratio=1.1369454455076426
Average over all trials: ss=3.69889 os=3.5371900000000003 Ratio=1.0457142534045385


## Random Digraph (Remove 0%)

In [39]:
rep_ss_ratio("random_digraph", topo_p=4.0, remove_p=0.0)

Trial 0: 1944 Steady states. Avg weight=1.8113 Oscil. w avg=11.8611 Ratio=0.1527092765426478
Trial 1: 1933 Steady states. Avg weight=1.6447 Oscil. w avg=11.5989 Ratio=0.1417979291139677
Trial 2: 1941 Steady states. Avg weight=1.4044 Oscil. w avg=10.9223 Ratio=0.12858097653424647
Trial 3: 1994 Steady states. Avg weight=1.6696 Oscil. w avg=12.3535 Ratio=0.1351519812198972
Trial 4: 1943 Steady states. Avg weight=1.5537 Oscil. w avg=11.3631 Ratio=0.13673205375293715
Trial 5: 1947 Steady states. Avg weight=1.8523 Oscil. w avg=11.5287 Ratio=0.16066859229574887
Trial 6: 1945 Steady states. Avg weight=1.8047 Oscil. w avg=11.5576 Ratio=0.1561483352945248
Trial 7: 1946 Steady states. Avg weight=1.6116 Oscil. w avg=11.4951 Ratio=0.14019886734347678
Trial 8: 1948 Steady states. Avg weight=1.8643 Oscil. w avg=11.2544 Ratio=0.1656507676997441
Trial 9: 1924 Steady states. Avg weight=1.8198 Oscil. w avg=11.4835 Ratio=0.15847084947968826
Average over all trials: ss=1.70364 os=11.541820000000001 Ratio=0

## Random Digraph (Remove 90%)

In [40]:
rep_ss_ratio("random_digraph", topo_p=4.0, remove_p=0.9)

Trial 0: 343 Steady states. Avg weight=4.0039 Oscil. w avg=3.3719 Ratio=1.1874314184880927
Trial 1: 359 Steady states. Avg weight=3.9225 Oscil. w avg=3.8571 Ratio=1.0169557439527106


/Users/djpassey/Code/ReservoirSpecialization/rescomp/ResComp.py:166: UserWarning: Spectral radius of reservoir is close to zero. Edge weights will not be scaled
  warn("Spectral radius of reservoir is close to zero. Edge weights will not be scaled")


Trial 2: 8 Steady states. Avg weight=6.6971 Oscil. w avg=5.0314 Ratio=1.3310609373136701


/Users/djpassey/Code/ReservoirSpecialization/rescomp/ResComp.py:166: UserWarning: Spectral radius of reservoir is close to zero. Edge weights will not be scaled
  warn("Spectral radius of reservoir is close to zero. Edge weights will not be scaled")


Trial 3: 7 Steady states. Avg weight=7.6326 Oscil. w avg=5.124 Ratio=1.4895784543325528
Trial 4: 337 Steady states. Avg weight=3.9181 Oscil. w avg=3.7598 Ratio=1.0421033033672005


...

Trial 5: 6 Steady states. Avg weight=3.7071 Oscil. w avg=4.9018 Ratio=0.756273205761149


/Users/djpassey/Code/ReservoirSpecialization/rescomp/ResComp.py:166: UserWarning: Spectral radius of reservoir is close to zero. Edge weights will not be scaled
  warn("Spectral radius of reservoir is close to zero. Edge weights will not be scaled")


Trial 6: 5 Steady states. Avg weight=5.616 Oscil. w avg=4.6534 Ratio=1.2068595005802207


/Users/djpassey/Code/ReservoirSpecialization/rescomp/ResComp.py:166: UserWarning: Spectral radius of reservoir is close to zero. Edge weights will not be scaled
  warn("Spectral radius of reservoir is close to zero. Edge weights will not be scaled")


Trial 7: 5 Steady states. Avg weight=6.2048 Oscil. w avg=4.2704 Ratio=1.4529786436867738
Trial 8: 347 Steady states. Avg weight=3.1013 Oscil. w avg=3.166 Ratio=0.9795641187618447
Trial 9: 366 Steady states. Avg weight=3.2968 Oscil. w avg=3.4678 Ratio=0.9506891977622701
Average over all trials: ss=4.81002 os=4.160359999999999 Ratio=1.156154755838437


## Erdos (Remove 0%)

In [41]:
rep_ss_ratio("erdos", topo_p=4.0, remove_p=0.0)

Trial 0: 1736 Steady states. Avg weight=1.8574 Oscil. w avg=10.3254 Ratio=0.1798864935014624
Trial 1: 1720 Steady states. Avg weight=1.8832 Oscil. w avg=10.7185 Ratio=0.17569622615104724
Trial 2: 1754 Steady states. Avg weight=1.7765 Oscil. w avg=9.4439 Ratio=0.18811084403689154
Trial 3: 1771 Steady states. Avg weight=1.6424 Oscil. w avg=10.2145 Ratio=0.1607910323559646
Trial 4: 1741 Steady states. Avg weight=1.823 Oscil. w avg=9.6866 Ratio=0.18819812937459995
Trial 5: 1722 Steady states. Avg weight=1.6077 Oscil. w avg=10.0591 Ratio=0.1598254316986609
Trial 6: 1757 Steady states. Avg weight=1.88 Oscil. w avg=10.2537 Ratio=0.183348449827867
Trial 7: 1742 Steady states. Avg weight=1.8897 Oscil. w avg=9.5842 Ratio=0.1971682560881451
Trial 8: 1766 Steady states. Avg weight=1.8079 Oscil. w avg=10.2661 Ratio=0.17610387586327816
Trial 9: 1782 Steady states. Avg weight=1.8347 Oscil. w avg=10.629 Ratio=0.17261266346787094
Average over all trials: ss=1.8002500000000001 os=10.1181 Ratio=0.1779237

## Erdos (Remove 90%)

In [42]:
rep_ss_ratio("erdos", topo_p=4.0, remove_p=0.9)

Trial 0: 278 Steady states. Avg weight=2.2313 Oscil. w avg=3.3875 Ratio=0.6586863468634686
Trial 1: 305 Steady states. Avg weight=3.295 Oscil. w avg=3.2999 Ratio=0.9985151065183793
Trial 2: 249 Steady states. Avg weight=2.574 Oscil. w avg=3.3587 Ratio=0.7663679399767768
Trial 3: 330 Steady states. Avg weight=2.7678 Oscil. w avg=3.3755 Ratio=0.8199674122352243
Trial 4: 306 Steady states. Avg weight=3.4646 Oscil. w avg=3.7082 Ratio=0.9343077503910252
Trial 5: 305 Steady states. Avg weight=2.7952 Oscil. w avg=3.5056 Ratio=0.7973528069374715
Trial 6: 289 Steady states. Avg weight=3.3988 Oscil. w avg=3.5212 Ratio=0.9652391230262412
Trial 7: 290 Steady states. Avg weight=2.5954 Oscil. w avg=3.3984 Ratio=0.7637123352165726
Trial 8: 222 Steady states. Avg weight=3.7313 Oscil. w avg=3.7405 Ratio=0.9975404357706189
Trial 9: 261 Steady states. Avg weight=3.3251 Oscil. w avg=3.0856 Ratio=1.077618615504278
Average over all trials: ss=3.01785 os=3.4381100000000004 Ratio=0.877764236746352


## Decrease Identity Spectral Radius

In [8]:
RES_PARAMS = {
    "uniform_weights": True,
    "solver": "ridge",
    "ridge_alpha": 1e-6,
    "signal_dim": 3,
    "network": "random graph",
    "res_sz": 2500,
    "activ_f": np.tanh,
    "connect_p": .1,
    "spect_rad": 1.0,
    "gamma": 5,
    "sigma": .14,
    "sparse_res": True,
}

rep_ss_ratio("ident", topo_p=None, remove_p=0.0)

Trial 0: 91 Steady states. Avg weight=5.748 Oscil. w avg=9.9653 Ratio=0.5768015012091959
Trial 1: 106 Steady states. Avg weight=5.457 Oscil. w avg=9.1449 Ratio=0.5967260440245382
Trial 2: 107 Steady states. Avg weight=6.5952 Oscil. w avg=9.9836 Ratio=0.6606033895588767
Trial 3: 108 Steady states. Avg weight=6.7404 Oscil. w avg=10.2401 Ratio=0.658235759416412


KeyboardInterrupt: 